In [1]:
import pandas as pd
import numpy as np



In [2]:
## read the data set
df=pd.read_csv("Advertising_data.csv")
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19.0,19000.0,0
1,15810944,Male,35.0,20000.0,0
2,15668575,Female,26.0,43000.0,0
3,15603246,Female,27.0,57000.0,0
4,15804002,Male,19.0,76000.0,0


In [3]:
X=df.iloc[:,[2,3]]
y=df.iloc[:,4]
print(X.head())
print(y.head())

    Age  EstimatedSalary
0  19.0          19000.0
1  35.0          20000.0
2  26.0          43000.0
3  27.0          57000.0
4  19.0          76000.0
0    0
1    0
2    0
3    0
4    0
Name: Purchased, dtype: int64


In [4]:
from sklearn.model_selection import train_test_split


In [5]:
## Hyperparameter using the  GridSearchCV
## Spliting the data into the train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)

In [6]:
## feature Scaling 
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)


In [7]:
# X_train ## the SCaled down values

from sklearn.svm import SVC
classifer=SVC(kernel='linear',random_state=0)
classifer.fit(X_train,y_train)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [8]:
y_pred=classifer.predict(X_test)

In [9]:
### check the accuracy with the above parameter
from sklearn.metrics import accuracy_score
ac=accuracy_score(y_test,y_pred)
ac ## It will give the accuracy value for above aparameter


## Now the questions come here , that how we can increase this accuracy. What are the parameters we can take to increase the accuarcy.
## Because in the reail scenario we have to find the best acouray for optimal parameter.
## 1. GridSearchCV
## 2. RandomSearchCV


0.85

In [10]:
## HyperParameter tuning
from sklearn.model_selection import GridSearchCV

paramaters=[{'C':[1,10,100,1000],'kernel':['linear']},
            {'C':[1,10,100,100],'kernel':['rbf'],'gamma':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}]

grid_search=GridSearchCV(estimator=classifer,
                         param_grid=paramaters,
                         scoring='accuracy',
                         cv=10,
                         n_jobs=-1)
              
grid_search=grid_search.fit(X_train,y_train)


In [11]:
grid_search.best_score_  ## It will give you the best score

0.9100000000000001

In [12]:
grid_search.best_params_ ## It will give the best paramater

{'C': 10, 'gamma': 0.3, 'kernel': 'rbf'}

In [13]:
## Now check the Accuracy of the SVM with the hyperparameter what we got above using the GridSearchCV
classifer_hyper=SVC(C=10,gamma=0.3,kernel='rbf',random_state=0)
classifer_hyper.fit(X_train,y_train)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.3, kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [14]:
y_pred_hyper=classifer_hyper.predict(X_test)

ac_hyper=accuracy_score(y_test,y_pred_hyper)
print("The Accuracy after the hyperparameter tuning",ac_hyper)
print("The Accuracy before the hyperparameter tuning",ac)

The Accuracy after the hyperparameter tuning 0.93
The Accuracy before the hyperparameter tuning 0.85


In [15]:
## RandomizedSearchCV
## it is bit fater than GridSearchCV


In [16]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [17]:
random_classifier=RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=50)
random_classifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=50, verbose=0,
                       warm_start=False)

In [25]:
y_pred_rnd=random_classifier.predict(X_test)

In [28]:
ac_rand=accuracy_score(y_test,y_pred_rnd) ## the accuracy with the random forest classifier
ac_rand

0.87

In [38]:
## hyperparameter tuning
from scipy.stats import randint
est=RandomForestClassifier(n_jobs=-1)
rf_param_dist={"max_depth":[3,5,10,None],
               'n_estimators':[10,100,200,300,400,500],
               'max_features':randint(1,3),
               'criterion':['gini','entropy'],
               'bootstrap':[True,False],
               'min_samples_leaf':randint(1,4)}

def hypertuning_rscv(est,p_dist,nbr_iter,X,y):
   rdmsearch=RandomizedSearchCV(est,param_distributions=p_dist,n_jobs=-1,n_iter=nbr_iter,cv=9)
   rdmsearch.fit(X,y)
   ht_params=rdmsearch.best_params_
   ht_score=rdmsearch.best_score_
   return ht_params,ht_score



In [39]:
ht_params_res,ht_score_res=hypertuning_rscv(est,rf_param_dist,40,X,y)

In [40]:
ht_params_res

{'bootstrap': True,
 'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 2,
 'min_samples_leaf': 3,
 'n_estimators': 200}

In [41]:
ht_score_res

0.9120650953984286

In [49]:
random_classifier=RandomForestClassifier(n_estimators=200,bootstrap=True,criterion='entropy',random_state=42,max_depth=10,max_features=2,min_samples_leaf=3)
random_classifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=10, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [50]:
y_pred_hp_rand=random_classifier.predict(X_test)

In [51]:
ac_rand=accuracy_score(y_test,y_pred_hp_rand) ## the accuracy with the random forest classifier
ac_rand ## Accuracy after hypertuning

0.89

In [18]:
initial_data = {'First_name': ['Ram', 'Mohan', 'Tina', 'Jeetu', 'Meera'],  
        'Last_name': ['Kumar', 'Sharma', 'Ali', 'Gandhi', 'Kumari'],  
        'Age': [42, 52, 36, 21, 23],  
        'City': ['Mumbai', 'Noida', 'Pune', 'Delhi', 'Bihar']} 

df=pd.DataFrame(initial_data,columns=['First_name','Last_name','Age','City'])

In [19]:
df

,First_name,Last_name,Age,City
0,Ram,Kumar,42,Mumbai
1,Mohan,Sharma,52,Noida
2,Tina,Ali,36,Pune
3,Jeetu,Gandhi,21,Delhi
4,Meera,Kumari,23,Bihar


In [20]:
# Create new column using dictionary 
new_data = { "Ram":"B.Com", 
             "Mohan":"IAS", 
             "Tina":"LLB", 
             "Jeetu":"B.Tech", 
             "Meera":"MBBS" } 
  

In [21]:
# Combine the new data with  the existing frame work
df['Qualification']=df['First_name'].map(new_data)
#df.drop(['Qualificatio'],axis=1,inplace=True) ## droping of the one ofthe  column

df

,First_name,Last_name,Age,City,Qualification
0,Ram,Kumar,42,Mumbai,B.Com
1,Mohan,Sharma,52,Noida,IAS
2,Tina,Ali,36,Pune,LLB
3,Jeetu,Gandhi,21,Delhi,B.Tech
4,Meera,Kumari,23,Bihar,MBBS


In [22]:
df

,First_name,Last_name,Age,City,Qualification
0,Ram,Kumar,42,Mumbai,B.Com
1,Mohan,Sharma,52,Noida,IAS
2,Tina,Ali,36,Pune,LLB
3,Jeetu,Gandhi,21,Delhi,B.Tech
4,Meera,Kumari,23,Bihar,MBBS


In [23]:
initial_datas = {'First_name': ['Ram', 'Mohan', 'Tina', 'Jeetu', 'Meera'],  
                'Last_name': ['Kumar', 'Sharma', 'Ali', 'Gandhi', 'Kumari'],  
                'Age': [42, 52, 36, 21, 23],  
                'City': ['Mumbai', 'Noida', 'Pune', 'Delhi', 'Bihar']} 
  
df = pd.DataFrame(initial_datas, columns = ['First_name', 'Last_name', 
                                                      'Age', 'City']) 
  
# Create new column using dictionary 
new_data = { 0:"Shyam", 
             2:"Riya", 
             3:"Jitender" } 
  
# combine this new data with existing DataFrame 
df["First_name"].update(pd.Series(new_data))
df

,First_name,Last_name,Age,City
0,Shyam,Kumar,42,Mumbai
1,Mohan,Sharma,52,Noida
2,Riya,Ali,36,Pune
3,Jitender,Gandhi,21,Delhi
4,Meera,Kumari,23,Bihar
